In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import zipfile
import tempfile
from tqdm import tqdm

In [2]:
# Define Alex net
class AlexNet(nn.Module):
    def __init__(self, num_classes=6):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [4]:
# Define data transformations
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create a temporary directory
temp_dir = tempfile.mkdtemp()

# Extract the contents of the zip file to the temporary directory
with zipfile.ZipFile('drive/MyDrive/others/dataset.zip', 'r') as zip_ref:
    zip_ref.extractall(temp_dir)

# Load the PlantVillage dataset
train_data = datasets.ImageFolder(root=temp_dir + '/dataset/train', transform=data_transforms)
test_data = datasets.ImageFolder(root=temp_dir + '/dataset/test', transform=data_transforms)

# Create data loaders for training and test sets
train_loader = DataLoader(train_data, batch_size=256, shuffle=True)
test_loader = DataLoader(test_data, batch_size=256)

# Define the model
model = AlexNet(num_classes=len(train_data.classes))

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [5]:
# Train the model using cross-validation
for epoch in range(2):
    running_loss = 0.0
    print(f'Training epoch {epoch+1}/{2}')
    for i, data in enumerate(tqdm(train_loader)):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

Training epoch 1/2


100%|██████████| 10/10 [04:05<00:00, 24.54s/it]


Training epoch 2/2


100%|██████████| 10/10 [04:10<00:00, 25.06s/it]


In [7]:
# Evaluate the model on the test set
with torch.no_grad():
    y_true = []
    y_pred = []
    for data in test_loader:
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        y_true.extend(labels.tolist())
        y_pred.extend(predicted.tolist())

# Calculate performance metrics
accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='macro')
precision = precision_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

# Print performance metrics
print(
    f'Epoch: {epoch} Loss: {running_loss / len(train_loader)} Accuracy: {accuracy} Recall: {recall} Precision: {precision} F1: {f1}\n\n')

Epoch: 1 Loss: 1.596327531337738 Accuracy: 0.4816 Recall: 0.2 Precision: 0.09631999999999999 F1: 0.13002159827213822




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
